# EBM Classifier (Microsoft)

Modelo de boosting interpretable basado en funciones aditivas generalizadas (GAM). La gracia de EBM (Explainable Boosting Machine) es su interpretabilidad ya que se desarrollo para eso. Es utilizado, por ejemplo, en áreas de la salud y en la banca.

Como funciona:

Cada función es aprendida como un conjunto de bins y árboles simples (reglas). Similar a un GAM pero las funciones se aprenden con boosting.

Obs: Boosting se refiere a que siempre va mejorando el error de la etapa anterior.

In [ ]:
!pip install -q interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 14.9 MB/s eta 0:00:00


Carguemos las librerías necesarias para implementar el modelo.

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Carguemos el dataset y dividamos los datos en entrenamiento y validación.

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
feature_names = data.feature_names

Entrenamiento.

In [ ]:
ebm = ExplainableBoostingClassifier(random_state=42, feature_names=feature_names)
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier(feature_names=array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23'))

Realicemos la predicción del modelo y evaluemos su rendimiento con las métricas.

In [ ]:
# Prediccion
pred_ebm = ebm.predict(X_test)

# Metricas
print(confusion_matrix(y_test, pred_ebm))
print(classification_report(y_test, pred_ebm))

[[38  4]
 [ 0 72]]
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        42
           1       0.95      1.00      0.97        72

    accuracy                           0.96       114
   macro avg       0.97      0.95      0.96       114
weighted avg       0.97      0.96      0.96       114



Ahora procesamos a explicar los resultados.

Muestra en orden de mayor a menor importancia cuanto es el aporte de cada una de las variables a la predicción.

Al seleccionar worst texture podemos ver como afecta a la prediccion.

Por ejemplo, para un valor de 15 en la worst texture, le suma 2 a la probabilidad de que sea 1 (cancer maligno).

Lo gris es la banda de error en la prediccion. La linea azul es la función que predice el valor que va a tomar la prediccion para esta variable (en este caso sigmoide).

In [ ]:
ebm_global = ebm.explain_global(name = 'EBM Breast Cancer (con nombres)')
show(ebm_global)

El modelo aprendió que valores bajos (15,20) de worst texture incrementan el score, es decir, hacen mas probable que un tumor sea maligno y valores altos (30,40) no contribuyen o bajan el score, lo que implica menos probabilidad de malignidad.

Es decir, el modelo confia mas cuando el valor es bajo y hay suficiente evidencia en los datos.